In [44]:
%reset
from sympy import *
import numpy as np
eta, deta, theta = symbols('eta, deta, theta')
eta_n, eta_s, n1, n2, Exx, Eyy, Exy =  symbols('eta_n, eta_s, n1, n2, Exx, Eyy, Exy')

In [45]:
E_mat = Matrix([[Exx, Exy], [Exy, Eyy]])
E     = Matrix([[Exx], [Eyy], [Exy]])
R     = Matrix(2,2,[n2,-n1,n1,n2]) 
E_rot_mat = R*E_mat*R.T 
display(E_rot_mat) # remains symmetric
E_rot     = Matrix([ [E_rot_mat[0,0]], [E_rot_mat[1,1]], [E_rot_mat[1,0]]])
D_rot     = Matrix([ [eta_n, 0, 0], [0, eta_n, 0], [0, 0, eta_s] ])
S_rot     = 2*D_rot*E_rot
S_rot_mat = Matrix([ [S_rot[0], S_rot[2]], [S_rot[2], S_rot[1] ] ] )
S_mat     = R.T*S_rot_mat*R
S         = Matrix([ [S_mat[0,0]], [S_mat[1,1]], [S_mat[1,0]]])
display(S) 

Matrix([
[-n1*(Exy*n2 - Eyy*n1) + n2*(Exx*n2 - Exy*n1), n1*(Exx*n2 - Exy*n1) + n2*(Exy*n2 - Eyy*n1)],
[-n1*(Exy*n1 + Eyy*n2) + n2*(Exx*n1 + Exy*n2), n1*(Exx*n1 + Exy*n2) + n2*(Exy*n1 + Eyy*n2)]])

Matrix([
[  n1*(2*eta_n*n1*(n1*(Exx*n1 + Exy*n2) + n2*(Exy*n1 + Eyy*n2)) + 2*eta_s*n2*(-n1*(Exy*n1 + Eyy*n2) + n2*(Exx*n1 + Exy*n2))) + n2*(2*eta_n*n2*(-n1*(Exy*n2 - Eyy*n1) + n2*(Exx*n2 - Exy*n1)) + 2*eta_s*n1*(-n1*(Exy*n1 + Eyy*n2) + n2*(Exx*n1 + Exy*n2)))],
[-n1*(-2*eta_n*n1*(-n1*(Exy*n2 - Eyy*n1) + n2*(Exx*n2 - Exy*n1)) + 2*eta_s*n2*(-n1*(Exy*n1 + Eyy*n2) + n2*(Exx*n1 + Exy*n2))) + n2*(2*eta_n*n2*(n1*(Exx*n1 + Exy*n2) + n2*(Exy*n1 + Eyy*n2)) - 2*eta_s*n1*(-n1*(Exy*n1 + Eyy*n2) + n2*(Exx*n1 + Exy*n2)))],
[ n1*(2*eta_n*n2*(n1*(Exx*n1 + Exy*n2) + n2*(Exy*n1 + Eyy*n2)) - 2*eta_s*n1*(-n1*(Exy*n1 + Eyy*n2) + n2*(Exx*n1 + Exy*n2))) + n2*(-2*eta_n*n1*(-n1*(Exy*n2 - Eyy*n1) + n2*(Exx*n2 - Exy*n1)) + 2*eta_s*n2*(-n1*(Exy*n1 + Eyy*n2) + n2*(Exx*n1 + Exy*n2)))]])

In [60]:
# Compute components by formulating the Jacobian
D_ani = Matrix(3,3,[0,0,0,0,0,0,0,0,0])
for i in range(3):
    for j in range(3):
        D_ani[i,j] = S[i].diff(E[j]).subs(eta_n, eta+deta).subs(eta_s, eta-deta).factor(eta,deta).simplify()
        # D_ani[i,j] = D_ani[i,j].subs(n1**4 - 2*n1**2*n2**2 + n2**4, (n1**2 - n2**2)**2)
        # D_ani[i,j] = D_ani[i,j].subs(n1**4 + 2*n1**2*n2**2 + n2**4, (n1**2 + n2**2)**2)
        # D_ani[i,j] = D_ani[i,j].subs((n1**2 + n2**2), 1)
        # D_ani[i,j] = D_ani[i,j].subs((n1**2 - n2**2), 2*n1**2-1)
display(D_ani)

print( octave_code(D_ani[0,0]) )


Matrix([
[2*deta*(n1**4 - 2*n1**2*n2**2 + n2**4) + 2*eta*(n1**4 + 2*n1**2*n2**2 + n2**4),                                                             8*deta*n1**2*n2**2,                                                    8*deta*n1*n2*(n1**2 - n2**2)],
[                                                            8*deta*n1**2*n2**2, 2*deta*(n1**4 - 2*n1**2*n2**2 + n2**4) + 2*eta*(n1**4 + 2*n1**2*n2**2 + n2**4),                                                   8*deta*n1*n2*(-n1**2 + n2**2)],
[                                                  4*deta*n1*n2*(n1**2 - n2**2),                                                  4*deta*n1*n2*(-n1**2 + n2**2), -2*deta*(n1**4 - 6*n1**2*n2**2 + n2**4) + 2*eta*(n1**4 + 2*n1**2*n2**2 + n2**4)]])

2*deta.*(n1.^4 - 2*n1.^2.*n2.^2 + n2.^4) + 2*eta.*(n1.^4 + 2*n1.^2.*n2.^2 + n2.^4)


In [47]:
# Check we recover isotropic
D_ani_11 = D_ani[0,0].simplify().subs(eta_n, eta+deta).subs(eta_s, eta-deta)
D_ani_33 = D_ani[2,2].simplify().subs(eta_n, eta+deta).subs(eta_s, eta-deta)
display(trigsimp(D_ani_11.subs(deta,0).factor(eta).simplify().subs(n1, cos(theta)).subs(n2, sin(theta))) )
display(trigsimp(D_ani_33.subs(deta,0).factor(eta).simplify().subs(n1, cos(theta)).subs(n2, sin(theta))) )


2*eta

2*eta